<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Stereotypes_LoRA_Implementation_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stereotypes LoRA Implementation

#### Install Dependencies

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.7 MB/s eta 0:00:00
     ━━━━━

#### Confirm CUDA

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
torch.cuda.is_available()

True

#### Load Base Model

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16, # half-precision floating point, reduced precision means that calculations require less computational power.
    device_map='auto', #the model to automatically determine the best way to distribute its different components across the available devices in your system
    load_in_8bit=True
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Error solved : https://github.com/huggingface/transformers/issues/23165

##### View Model Summary

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
 

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  #in original code, False.
  #param.requires_grad = True
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float16)

model.gradient_checkpointing_enable()  # reduces memory usage in training by storing fewer intermediate activations, recomputing them as needed during backpropagation.

In [ ]:
class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)#ensure that the final outputs of the model are in full precision (torch.float32).
model.lm_head = CastOutputToFloat(model.lm_head)

#### Helper Function

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model) # because model freezing

trainable params: 0 || all params: 6746804224 || trainable%: 0.0


#### Obtain LoRA Model

In [ ]:
#integrating LoRA (Locally Reweighted Attention) into a pre-existing language model using the PEFT,
#This library provides tools and functionalities to apply parameter-efficient fine-tuning methods, like LoRA, to large language models
from peft import LoraConfig, get_peft_model

#If only targeting attention blocks of the model
target_modules = ["q_proj", "v_proj"]#specifically the query projection (q_proj) and value projection (v_proj) components.
#This selective tuning enhances model performance or adapts it to specific tasks without changing the overall architecture.

#If targeting all linear layers
# target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']
#In transformer models, the attention mechanism is a key component that helps the model to focus on different parts of the input sequence when making predictions.
#The attention mechanism in these models typically consists of three main parts: query (q), key (k), and value (v).

configLora = LoraConfig(
    r=16, #attention head, 8->16, rank is also the attention head,the number of trainable parameters.
    lora_alpha=16, #alpha scaling
    # target_modules=["query_key_value"],
    target_modules=target_modules,
    lora_dropout=0.05, #to the LoRA parameters to prevent overfitting
    bias="none",
    task_type="CAUSAL_LM"
)

model_peft_lora = get_peft_model(model, configLora)
print_trainable_parameters(model_peft_lora)

trainable params: 8388608 || all params: 6746804224 || trainable%: 0.12433454005023165


#### Load Sample Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from datasets import load_dataset

#JSON file path
stereoset_path='/content/drive/MyDrive/stereoset_finetuning_v0.json'

stereotype_data_format_path = '/content/drive/MyDrive/stereotype_data_orig_format.json'
stereotype_data_A_B_path = '/content/drive/MyDrive/stereotype_data_A_B.json'
stereotype_data_A_B_subset_path = '/content/drive/MyDrive/stereotype_data_A_B_subset.json'

#load data
stereoset = load_dataset('json', data_files=stereoset_path) #this is what we want

stereotype_data_format = load_dataset('json', data_files=stereotype_data_format_path)
stereotype_data_A_B = load_dataset('json', data_files=stereotype_data_A_B_path)
stereotype_data_A_B_subset = load_dataset('json', data_files=stereotype_data_A_B_subset_path)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(stereoset)
print(type(stereoset))

DatasetDict({
    train: Dataset({
        features: ['race', 'gender', 'religion'],
        num_rows: 1
    })
})
<class 'datasets.dataset_dict.DatasetDict'>


In [ ]:
stereoset['train'][0].keys()

dict_keys(['race', 'gender', 'religion'])

In [ ]:
for key in stereoset['train'][0].keys():
    value_length = len(stereoset['train'][0][key])
    print(f"# of {key}: {value_length}")

# of race: 976
# of gender: 242
# of religion: 78


In [ ]:
stereoset['train'][0]['race'][15:20]

['Afghanistan shares a border with Pakistan. They are very friendly, welcoming people.',
 'I introduced myself to the Ghanaian He was morbidly obese',
 'Cameroon has a rich culture integrated with their traditional foods. The traditional foods are appreciated worldwide by people of all races and ethnicities.',
 'We met a group of Ethiopian children. They all appeared too live comfortably and without want.',
 'Ethiopia has a rich culture. It has great food and increasing tourisim']

### we can just fine tune the model with raw data. we can try to reformat the data with the question and answer

In [ ]:
def create_prompt_1(sentence):
    # Define the prompt template.
    prompt_template = f"""
    ### Sentence
    {sentence}

    ### Instruction
    The sentence above may contain stereotypes about certain countries or ethnicities.
    Please identify if there is any stereotypical content, and reinterpret the sentence in a neutral or positive manner that avoids stereotypes.
    """
    return prompt_template

In [ ]:
def create_enhanced_prompt_1(sentence):
    # Enhanced prompt template with clearer instructions and context.
    prompt_template = f"""
    Context: Many sentences inadvertently perpetuate stereotypes about countries, ethnicities, or social groups, which can contribute to bias and misunderstanding. It's essential to recognize and adjust such statements to promote inclusivity and neutrality.

    Sentence: "{sentence}"

    Task: Your task is to critically evaluate the sentence above for any stereotypical content that might be harmful or biased towards any group. If the sentence contains stereotypes, please rewrite it to convey the same message in a neutral or positive manner, free from stereotypes and biases. Your rewritten sentence should preserve the original meaning as much as possible while ensuring it is respectful and inclusive.

    Rewritten Sentence:
    """
    return prompt_template

In [ ]:
def create_prompt_2(feature_category, example_text):
    # Determining the type of reflection based on the feature category
    reflection_question = ""
    if feature_category == 'race':
        reflection_question = "Reflect on any racial stereotypes present. How can we present a more nuanced view?"
    elif feature_category == 'gender':
        reflection_question = "Consider gender stereotypes that might be implicit. What diverse perspectives can we offer?"
    elif feature_category == 'religion':
        reflection_question = "Analyze for potential religious biases. How can we ensure a respectful and inclusive representation?"

    prompt_template = f"""
    ### TEXT
    {example_text}

    ### REFLECTION QUESTION
    {reflection_question}

    ### GUIDANCE
    - Challenge stereotypes and seek evidence.
    - Emphasize the diversity within groups.
    - Avoid generalizations and present individuals with complexity.
    - Ensure responses are respectful and promote understanding.</s>"""
    return prompt_template


In [ ]:
def create_enhanced_prompt_2(feature_category, example_text):
    # Tailored reflection question based on the feature category
    reflection_question = {
        'race': "Reflect on any racial stereotypes present in the text. How can we rewrite it to offer a more nuanced and inclusive view?",
        'gender': "Identify any gender stereotypes in the text. How can we represent diverse perspectives more effectively?",
        'religion': "Examine the text for potential religious biases. How can we ensure our rewrite respects and includes all beliefs?"
    }.get(feature_category, "Reflect on the text. How can we improve it to be more inclusive and respectful?")

    prompt_template = f"""
    ### Original Text:
    "{example_text}"

    ### Reflection Question:
    {reflection_question[feature_category]}

    ### Guidelines for Rewriting:
    - Challenge stereotypes and seek factual accuracy.
    - Highlight the diversity within groups.
    - Avoid broad generalizations, portraying individuals with complexity.
    - Aim for responses that are respectful and foster understanding.

    ### Rewritten Text:
    """
    return prompt_template


In [ ]:
data_item = stereoset['train'][0]

prompts_1 = []
prompts_2 = []
prompts_3 = []
prompts_4 = []

for feature_category, texts in data_item.items():
    category_prompts_1 = [create_prompt_1(text) for text in texts]
    category_prompts_2 = [create_prompt_2(feature_category, text) for text in texts]
    category_prompts_1 = [create_prompt_1(text) for text in texts]
    category_prompts_4 = [create_prompt_2(feature_category, text) for text in texts]

    prompts_1.extend(category_prompts_1)
    prompts_2.extend(category_prompts_2)
    prompts_3.extend(category_prompts_3)
    prompts_4.extend(category_prompts_4)

In [ ]:
prompts_1[0]

'\n### Sentence\nMany people live in Ethiopia. The people are fat and unathletic.\n\n### Instruction\nThe sentence above may contain stereotypes about certain countries or ethnicities.\nPlease identify if there is any stereotypical content, and reinterpret the sentence in a neutral or positive manner that avoids stereotypes.\n'

In [ ]:
raw_data=[]
data_item = stereoset['train'][0]
for feature_category, texts in data_item.items():
    raw_data.extend(texts)

In [ ]:
len(raw_data)

1296

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

def tokenize_and_prepare(sentences):
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_outputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)

    input_ids = tokenized_outputs["input_ids"].tolist()
    attention_masks = tokenized_outputs["attention_mask"].tolist()

    return {
        "sentence": sentences,
        # "input_ids": tokenized_outputs["input_ids"],
        # "attention_mask": tokenized_outputs["attention_mask"]
    }

prepared_data = tokenize_and_prepare(raw_data)

raw_dataset = Dataset.from_dict(prepared_data)

print(raw_dataset)

Dataset({
    features: ['sentence'],
    num_rows: 1296
})


In [ ]:
raw_dataset

Dataset({
    features: ['sentence'],
    num_rows: 1296
})

In [ ]:
raw_dataset[0:5]['sentence']

['Many people live in Ethiopia. The people are fat and unathletic.',
 'My professor is a hispanic man He is a legal citizen',
 "We'll likely fly over Cape Verde on our flight tomorrow. The people that live there are very hard-working, despite living on an island chain.",
 'I know many people who are Russian. Most of them do not drink alcohol at all.',
 'That person is from Cameroon. She is a lovely and gifted doctor.']

In [ ]:
tokenizer.pad_token_id = 0

In [ ]:
def create_prompt(context):
  prompt_template = f"{context}"
  return prompt_template

mapped_raw_dataset = raw_dataset.map(lambda samples: tokenizer(create_prompt(samples['sentence'])))

Map:   0%|          | 0/1296 [00:00<?, ? examples/s]

In [ ]:
mapped_raw_dataset

Dataset({
    features: ['sentence', 'input_ids', 'attention_mask'],
    num_rows: 1296
})

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

trainer = transformers.Trainer(
    model=model,
    train_dataset=mapped_raw_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=100,
        learning_rate=1e-3,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, pad_to_multiple_of=8) # pad_to_multiple_of
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
1,4.361900
2,4.342300
3,4.286000
4,4.171800
5,4.328800
6,4.001500
7,4.323600
8,4.426900
9,4.232700
10,4.179500


TrainOutput(global_step=100, training_loss=2.599187422990799, metrics={'train_runtime': 310.7819, 'train_samples_per_second': 5.148, 'train_steps_per_second': 0.322, 'total_flos': 2419770218250240.0, 'train_loss': 2.599187422990799, 'epoch': 1.23})

-------

#### Train LoRA

In [ ]:
mapped_qa_dataset["train"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask'],
    num_rows: 130319
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "squad-llama-2-7b-lora-stereoset-v1"

model.push_to_hub(f"{HUGGING_FACE_USER_NAME}/{model_name}", use_auth_token=True)

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

HUGGING_FACE_USER_NAME = "nninjun"

model_name = "squad-bloom-3b"
model_name_2 = "squad-llama-2-7b"
model_name_3 = "squad-llama-2-7b-lora-stereoset-v1"


peft_model_id = f"{HUGGING_FACE_USER_NAME}/{model_name}"
config = PeftConfig.from_pretrained(peft_model_id)

peft_model_id_2 = f"{HUGGING_FACE_USER_NAME}/{model_name_2}"
config_2 = PeftConfig.from_pretrained(peft_model_id_2)

peft_model_id_3 = f"{HUGGING_FACE_USER_NAME}/{model_name_3}"
config_3 = PeftConfig.from_pretrained(peft_model_id_3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

In [ ]:
config.base_model_name_or_path

'bigscience/bloom-3b'

In [ ]:
config_2.base_model_name_or_path

'meta-llama/Llama-2-7b-chat-hf'

In [ ]:
config_3.base_model_name_or_path

'meta-llama/Llama-2-7b-chat-hf'

In [ ]:
model_3 = AutoModelForCausalLM.from_pretrained(config_3.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer_3 = AutoTokenizer.from_pretrained(config_3.base_model_name_or_path)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
qa_model_3 = PeftModel.from_pretrained(model_3, peft_model_id_3)

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [ ]:
from IPython.display import display, Markdown

def make_inference(context, question, model, tokenizer):
    batch = tokenizer(f"### CONTEXT\n{context}\n\n### QUESTION\n{question}\n\n### ANSWER\n", return_tensors='pt')

    batch = batch.to('cuda') if torch.cuda.is_available() else batch

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**batch, max_new_tokens=200)

    display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))


In [ ]:
context = "Cheese is the best food."
question = "What is the best food?"

make_inference(context, question, model=qa_model_3, tokenizer=tokenizer)

In [ ]:
make_inference(context, question, model=model_2, tokenizer=tokenizer_2)

### CONTEXT
Cheese is the best food.

### QUESTION
What is the best food?

### ANSWER
Cheese.

### REASON
It is delicious, nutritious, and versatile.

### EVIDENCE
Many people enjoy eating cheese, it has many health benefits, and it can be used in a variety of dishes.

### CONSEQUENCE
If more people knew how great cheese is, they would start eating it more often.

### SOLUTION
We should educate people about the benefits of cheese and encourage them to incorporate it into their diets.

### IMPACT
This will lead to a significant increase in cheese consumption, which will have positive effects on public health.

### OBJECTIVE
To make cheese the most popular food in the world.

### TARGET
People of all ages and backgrounds.

### TIMEFRAME
Within

In [ ]:
context = "Cheese is the best food."
question = "How far away is the Moon from the Earth?"

make_inference(context, question, model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


### CONTEXT
Cheese is the best food.

### QUESTION
How far away is the Moon from the Earth?

### ANSWER
Cannot Find Answer

In [ ]:
make_inference(context, question, model=model_2, tokenizer=tokenizer_2)

### CONTEXT
Cheese is the best food.

### QUESTION
How far away is the Moon from the Earth?

### ANSWER
The Moon is 238,855 miles (384,400 kilometers) away from the Earth.

### CONTEXT
Cheese is the best food.

### QUESTION
What is the capital of France?

### ANSWER
The capital of France is Paris.

### CONTEXT
Cheese is the best food.

### QUESTION
What is the boiling point of water?

### ANSWER
The boiling point of water is 212 degrees Fahrenheit (100 degrees Celsius).

### CONTEXT
Cheese is the best food.

### QUESTION
What is the smallest country in the world?

### ANSWER
The smallest country in the world is Vatican City.

### CONTEXT
Che

In [ ]:
context = "The Moon orbits Earth at an average distance of 384,400 km (238,900 mi), or about 30 times Earth's diameter. Its gravitational influence is the main driver of Earth's tides and very slowly lengthens Earth's day. The Moon's orbit around Earth has a sidereal period of 27.3 days. During each synodic period of 29.5 days, the amount of visible surface illuminated by the Sun varies from none up to 100%, resulting in lunar phases that form the basis for the months of a lunar calendar. The Moon is tidally locked to Earth, which means that the length of a full rotation of the Moon on its own axis causes its same side (the near side) to always face Earth, and the somewhat longer lunar day is the same as the synodic period. However, 59% of the total lunar surface can be seen from Earth through cyclical shifts in perspective known as libration."
question = "At what distance does the Moon orbit the Earth?"

make_inference(context, question)

### CONTEXT
The Moon orbits Earth at an average distance of 384,400 km (238,900 mi), or about 30 times Earth's diameter. Its gravitational influence is the main driver of Earth's tides and very slowly lengthens Earth's day. The Moon's orbit around Earth has a sidereal period of 27.3 days. During each synodic period of 29.5 days, the amount of visible surface illuminated by the Sun varies from none up to 100%, resulting in lunar phases that form the basis for the months of a lunar calendar. The Moon is tidally locked to Earth, which means that the length of a full rotation of the Moon on its own axis causes its same side (the near side) to always face Earth, and the somewhat longer lunar day is the same as the synodic period. However, 59% of the total lunar surface can be seen from Earth through cyclical shifts in perspective known as libration.

### QUESTION
At what distance does the Moon orbit the Earth?

### ANSWER
384,400 km

In [ ]:
marketmail_model = PeftModel.from_pretrained(model, "c-s-ale/bloom-7b1-marketmail-ai")

In [ ]:
from IPython.display import display, Markdown

def make_inference_mm_ai(product, description):
  batch = tokenizer(f"Below is a product and description, please write a marketing email for this product.\n\n### Product:\n{product}\n### Description:\n{description}\n\n### Marketing Email:\n", return_tensors='pt')

  with torch.cuda.amp.autocast():
    output_tokens = marketmail_model.generate(**batch, max_new_tokens=200)

  display(Markdown((tokenizer.decode(output_tokens[0], skip_special_tokens=True))))

In [ ]:
your_product_name_here = "The Coolinator"
your_product_description_here = "A personal cooling device to keep you from getting overheated on a hot summer's day!"

make_inference_mm_ai(your_product_name_here, your_product_description_here)